In [1]:
%config Completer.use_jedi = False
import sys

sys.path.insert(0, './../')
import io
import requests
import pandas as pd
import dask.dataframe as dd

from core.data_acquisition import TaxiData
from copy import deepcopy

clean_df = deepcopy(TaxiData.clean_df)

### NYC Taxi Data - [Data Page](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)

---

Yellow Taxi File

In [6]:
def read_chunk(url: str, n_kb: int=int(1e5)) -> pd.DataFrame:
    """
    Read a chunk of data from url in kb
    """
    tries = 0
    while True:
        try:
            with requests.get(url, stream=True) as response:
                chunk = next(response.iter_content(chunk_size=n_kb))
            break
        except Exception as exc:
            tries += 1
            if tries > 5: 
                raise exc
    return pd.read_csv(io.BytesIO(chunk), error_bad_lines=False, sep=',')

url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-02.csv'
df1 = read_chunk(url)
print(df1.shape)

df1.head()

(544, 18)


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2010-02-13 01:46:36,2010-02-13 01:50:06,1,0.5,-73.967576,40.756357,1,0.0,-73.961301,40.760596,CRE,4.1,0.5,0.5,1.00,0.0,6.10
1,CMT,2010-02-13 21:50:04,2010-02-13 21:54:16,2,0.6,-73.970891,40.757594,1,0.0,-73.962648,40.761484,CAS,4.5,0.5,0.5,0.00,0.0,5.50
2,CMT,2010-02-12 23:32:17,2010-02-12 23:51:48,1,5.8,-73.940614,40.711774,1,0.0,-73.983736,40.672331,NA,16.9,0.5,0.5,0.00,0.0,17.90
3,CMT,2010-02-13 05:09:37,2010-02-13 05:14:39,1,1.2,-74.001726,40.741344,1,0.0,-73.988582,40.731488,NA,5.3,0.5,0.5,0.00,0.0,6.30
4,CMT,2010-02-13 16:11:05,2010-02-13 16:21:00,1,2.2,-73.995935,40.734535,1,0.0,-74.001672,40.756476,CRE,8.1,0.0,0.5,1.29,0.0,9.89
